In [ ]:
pip install rasterio # type: ignore;

: 

In [ ]:
pip install arcpy;

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime
import rasterio
import numpy as np

# CSV 파일 경로 설정
csv_file = r"E:\2025_Jeju_SectorCoupling\2024_Jeju_Solar.csv"
hillshade_dirs = {
    '제주': r"E:\2025_Jeju_SectorCoupling\SolarArea_Jeju",
    '고산': r"E:\2025_Jeju_SectorCoupling\SolarArea_Gosan",
    '서귀포': r"E:\2025_Jeju_SectorCoupling\SolarArea_Seoguipo"
}
output_dirs = {
    '제주': r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju",
    '고산': r"E:\2025_Jeju_SectorCoupling\PVgen_Gosan",
    '서귀포': r"E:\2025_Jeju_SectorCoupling\PVgen_Seoguipo"
}

# CSV 파일 로드
df = pd.read_csv(csv_file, encoding='utf-8')

# 일시 형식 변환 함수
def convert_timestamp(timestamp):
    return datetime.strptime(timestamp.replace("-", ":"), "%Y-%m-%d %H:%M:%S")

# 파일명에서 datetime 추출 함수
def extract_datetime(filename):
    # 모든 숫자를 추출
    extracted_numbers = re.findall(r'\d', filename)
    
    # 14자리 숫자가 있는지 확인
    if len(extracted_numbers) >= 14:
        result = ''.join(extracted_numbers[:14])  # 처음부터 14번째 숫자까지 이어 붙임
    else:
        print("Error:Wrong Filename")
        return None  # 숫자가 충분하지 않을 경우 종료

    # 날짜와 시간을 분리 (result를 사용)
    year = result[:4]
    month = result[4:6]
    day = result[6:8]
    hour = result[8:10]
    minute = result[10:12]
    second = result[12:14]

    # 원하는 포맷으로 결합
    formatted_datetime = f"{year}-{month}-{day} {hour}:{minute}:{second}"
    datetime_object = datetime.strptime(formatted_datetime, "%Y-%m-%d %H:%M:%S")
    print("datetime 객체:", datetime_object)
    return datetime_object

# 각 행을 반복 처리
for _, row in df.iterrows():
    location, timestamp = row[1], row[2]
    timestamp = convert_timestamp(timestamp)  # CSV의 일시를 datetime으로 변환
    
    if location not in hillshade_dirs:
        continue

    # 해당 지점의 Hillshade 파일 가져오기
    raster_files = [f for f in os.listdir(hillshade_dirs[location]) if f.endswith('.tif')]
    
    for raster_file in raster_files:
        raster_time = extract_datetime(raster_file)
        
        # 일시가 일치하지 않으면 넘어가기
        if raster_time != timestamp:
            continue
        
        # 래스터 파일 열기
        raster_path = os.path.join(hillshade_dirs[location], raster_file)
        with rasterio.open(raster_path) as src:
            # 셀 영역 계산
            raster_data = src.read(1)
            
            # 일사량 반영
            irradiance = row[3]
            
            # Hillshade가 null인 경우 결과도 null로 설정
            hillshade_factor = np.where(np.isnan(raster_data), 0, 1)
            
            # 발전량 계산
            solar_generation = np.where(hillshade_factor == 1, irradiance * 0.22 * 900 * hillshade_factor, np.nan)
            
            # 결과를 새로운 래스터 파일로 저장
            output_filename = f"PVgen_{location}_{timestamp.strftime('%Y-%m-%d %H-%M')}.tif"
            output_path = os.path.join(output_dirs[location], output_filename)

            # 저장할 래스터 파일 만들기
            with rasterio.open(output_path, 'w', driver='GTiff', 
                               count=1, dtype=solar_generation.dtype,
                               crs=src.crs, transform=src.transform,
                               width=src.width, height=src.height) as dst:
                dst.write(solar_generation, 1)
            
            # 파일이 저장되었음을 알림
            print(f"파일이 저장되었습니다: {output_path}")
